# OSRM (Open Source Routing Machine)

## Table of Contents

- [Introduction to the Match Service](#introduction-to-the-match-service)
- [Match Service](#match-service)
- [Loading the GPS Data](#loading-the-gps-data)
  - [Defining the GPS Point Structure](#defining-the-gps-point-structure)
  - [Parsing the GPX File](#parsing-the-gpx-file)
- [Plotting GPS Data](#plotting-gps-data)
  - [Visual Inspection of Raw GPS Points](#visual-inspection-of-raw-gps-points)
- [Using the Service](#using-the-service)
  - [General Options](#general-options)
  - [Additional Options](#additional-options)
  - [Making the Request](#making-the-request)
  - [No Radiuses](#no-radiuses)
  - [Extracting the Matched Geometry](#extracting-the-matched-geometry)
  - [Extracting the Snapped Route](#extracting-the-snapped-route)
  - [Visualizing the Matched Route](#visualizing-the-matched-route)
  - [Visualizing Route Legs](#visualizing-route-legs)
  - [With Radiuses](#with-radiuses)
- [Conclusion](#conclusion)

## Introduction to the Match Service

The OSRM **Match** service is designed to align raw GPS traces
with the most plausible path on the road network.

Unlike the Nearest service (which snaps individual points),
the Match service reconstructs an entire trajectory while
preserving temporal order and movement continuity.

This makes it ideal for:

- GPS trajectory correction  
- Vehicle trip reconstruction  
- Noisy signal alignment  
- Map-based movement analysis 

## Match Service

The OSRM **Match** service is used to align a sequence of GPS
coordinates with the most plausible path on the road network.

Unlike the Nearest service, which snaps individual points,
the Match service reconstructs a continuous route while
preserving movement order and trajectory logic.

It is particularly useful for:

- Correcting noisy GPS traces  
- Reconstructing vehicle trips  
- Aligning recorded movement data to roads  
- Performing trajectory analysis  

## Loading the GPS Data

To demonstrate the Match service, we begin by working with
a GPS trace file in **GPX format**.

You can download public GPS traces and ensure that the file
lies within the geographic area covered by your OSRM dataset
(e.g., downloaded from Geofabrik).

Each GPS track point contains:

- Latitude  
- Longitude  
- Timestamp (optional but recommended for better matching)  

### Defining the GPS Point Structure

We define a structured `Point` model to store:

- `latitude`  
- `longitude`  
- `timestamp` (Unix time)

This ensures that GPS data is handled in a clean,
structured format before being processed by OSRM.

In [ ]:
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float
    timestamp: int  # Unix timestamp

### Parsing the GPX File

The `parse_gpx_and_extract_points` function:

- Reads the GPX file using XML parsing  
- Extracts track points (`trkpt` elements)  
- Retrieves latitude and longitude attributes  
- Converts them into `Point` objects  
- Returns a list of GPS points  

This step transforms raw GPX XML data into a
usable Python data structure for routing operations.

In [ ]:
from pydantic import BaseModel

class Point(BaseModel):
    latitude: float
    longitude: float
    timestamp: int | None = None

Before running the following code, make sure to download a GPX trace
for your region and place it inside your project directory.

In my case, I downloaded a GPX trace for **Bremen** and saved it as:

`./data/gps_trace.gpx`

You can download public GPS trace files from:

https://www.openstreetmap.org/traces

After downloading the GPX file for your desired region,
update the `file_path` variable accordingly before executing the code.

In [ ]:
import xml.etree.ElementTree as ET

class Point:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude


def parse_gpx_and_extract_points(file_path: str) -> list[Point]:
    tree = ET.parse(file_path)
    root = tree.getroot()

    ns = {"default": "http://www.topografix.com/GPX/1/1"}

    points = []
    for trkpt in root.findall(".//default:trkpt", ns):
        lat = float(trkpt.attrib["lat"])
        lon = float(trkpt.attrib["lon"])

        points.append(Point(latitude=lat, longitude=lon))

    return points

file_path = "../data/gps_trace.gpx"
points = parse_gpx_and_extract_points(file_path)
len(points)

## Plotting GPS Data

After parsing the GPX file, i extracted **1004 GPS points**.

However, the OSRM Match service imposes a limit on the number
of coordinates that can be processed in a single request
(default is typically 100 points per call).

To stay within this constraint, we select a subset of points
for visualization and matching. The limit can be modified
in the OSRM configuration, but we will work within the
default constraints for this example.

### Visual Inspection of Raw GPS Points

Once plotted on the map, we observe that some GPS points
do not lie perfectly on the road network.

This occurs because:

- GPS signals are not perfectly accurate  
- Urban environments may cause signal reflections  
- Device precision may vary  
- Sampling intervals may introduce noise  

As a result, some points appear slightly offset
from the actual road geometry.

This is precisely why we use the **Match service** —
to reconstruct the most plausible route and correct
these spatial inconsistencies.

In [ ]:
import folium
from folium import Map


def get_folium_map(center_point: Point, points: list[Point], zoom_level: int = 14) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)

    for point in points:
        folium.CircleMarker(
            location=[point.latitude, point.longitude],
            radius=3,
            color='blue',
            fill=True,
            fill_color='blue'
        ).add_to(folium_map)

    return folium_map


max_points_to_use = 100
folium_map = get_folium_map(
    center_point=points[0],
    points=points[:max_points_to_use],
    zoom_level=18
)
folium_map

## Using the Service

### General Options

The Match service follows the same core parameter structure
as other OSRM services. Below is a summary of the required
parameters:

| Parameter   | Description |
|-------------|------------|
| `service`   | Specifies the OSRM operation. For this notebook, we use `match`. |
| `version`   | API version of the service. For OSRM 5.x installations, this is typically `v1`. |
| `profile`   | Routing profile used for matching (e.g., `driving`, `bike`, `foot`). |
| `coordinates` | Sequence of GPS points formatted as `{longitude},{latitude};{longitude},{latitude};...` |
| `format`    | Output format. Only `json` is supported and used by default. |

### Additional Options

The Match service also supports extra parameters
to improve matching accuracy:

| Option        | Description |
|--------------|------------|
| `timestamps` | UNIX timestamps (in seconds) for each coordinate. Must be strictly increasing. Helps OSRM preserve trajectory order. |
| `radiuses`   | Standard deviation of GPS precision (in meters). Improves matching robustness when GPS accuracy varies. |

### Making the Request

The Match endpoint follows this structure:

`/match/v1/{profile}/{coordinates}?steps={true|false}&geometries={polyline|polyline6|geojson}&overview={simplified|full|false}&annotations={true|false}`

Because we extracted 1004 GPS points, we must limit the
number of coordinates per request to stay within OSRM’s
processing constraints.

### No Radiuses

In this first example, we send the coordinates
without specifying radiuses.

We select a subset of the GPS points to ensure
the request remains within acceptable limits.

In [ ]:
service = 'match'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

In [ ]:
import requests

points_portion = points[:80]  # keep it small

coordinates = ";".join(
    f"{p.longitude},{p.latitude}" for p in points_portion
)

url = f"{host}/{service}/{version}/{profile}/{coordinates}"

params = {
    "geometries": "geojson",
    "overview": "full",
    "tidy": "true"
}

response = requests.get(url, params=params)
response.status_code

The request returned a `200` status code, which confirms
that the map matching operation was successful.

The response contains several fields, but the most important
one for us is:

- `matchings` → Contains the reconstructed route(s)
- `confidence` → Indicates how confident OSRM is about the match
- `legs` → Segments of the matched route
- `distance` → Total matched distance (in meters)
- `duration` → Total travel duration (in seconds

The snapped route geometry can be extracted from:

`response.json()["matchings"][0]`

In most cases, OSRM returns a single matching route,
so we access the first element in the `matchings` list.

In [ ]:
response.json()

### Extracting the Matched Geometry

Each matching contains a `geometry` field
that represents the reconstructed route aligned
to the road network.

Because we requested `geometries="geojson"`,
the coordinates are returned in:

`[longitude, latitude]` format.

To visualize them with Folium, we must convert them to:

`[latitude, longitude]`

### Extracting the Snapped Route

The matched route is available inside the `matchings` field
of the response object.

Since the Match service typically returns a single reconstructed
route, we access the first element:

`snapped_route["matchings"][0]`

In [ ]:
snapped_route = response.json()
snapped_route.keys()

In [ ]:
matched_route = snapped_route["matchings"][0]
matched_route.keys()

The returned matching contains several useful attributes:

- `confidence` → Confidence score of the match
- `geometry` → Reconstructed route aligned to the road network
- `legs` → Segments of the matched route
- `distance` → Total matched distance (meters)
- `duration` → Total matched duration (seconds)
- `weight` → Cost based on the routing profile

### Visualizing the Matched Route

To visualize the snapped route:

1. We first plot the original raw GPS points (blue markers).
2. Then we extract the full matched geometry from the response.
3. Since OSRM returns coordinates as `[longitude, latitude]`,
   we convert them to `[latitude, longitude]` for Folium.
4. Finally, we draw the reconstructed route as a red polyline.

In [ ]:
# Create base map with raw GPS points (blue dots)
folium_map = get_folium_map(
    center_point=points[0],
    points=points_portion,
    zoom_level=18
)

# Extract FULL snapped route geometry from OSRM Match response
geometry = matched_route["geometry"]["coordinates"]

# OSRM gives [lon, lat] → Folium needs [lat, lon]
snapped_points = [
    [lat, lon] for lon, lat in geometry
]

# Draw the snapped route (RED LINE)
folium.PolyLine(
    locations=snapped_points,
    color="red",
    weight=5,
    opacity=0.9,
    tooltip="OSRM Matched Route"
).add_to(folium_map)

# Show map
folium_map

The resulting red line represents the cleaned and reconstructed
trajectory generated by OSRM.

Notice how the route follows the road network smoothly,
even in areas where the raw GPS points slightly deviated
from the street. The Match service effectively removes
noise and produces a plausible driving path.

### Visualizing Route Legs

In this visualization, the matched route is split
according to its individual `legs`.

Each leg represents a segment of the overall route,
typically defined between major waypoints.

To make this segmentation clear:

- The first leg is drawn in **red**
- The second leg is drawn in **black**

Using different colors helps distinguish how the
complete matched route is internally structured.

In [ ]:
# Base map
folium_map = get_folium_map(
    center_point=points[0],
    points=points_portion,
    zoom_level=18
)

# Colors per leg
colors = ["red", "black"]

# Full snapped geometry (THIS EXISTS)
full_coords = matched_route["geometry"]["coordinates"]  # [lon, lat]
full_latlon = [[lat, lon] for lon, lat in full_coords]

legs = matched_route["legs"]

# Split geometry by leg distance
total_distance = sum(leg["distance"] for leg in legs)
cursor = 0

for i, leg in enumerate(legs):
    leg_color = colors[i % len(colors)]

    ratio = leg["distance"] / total_distance
    count = int(len(full_latlon) * ratio)

    segment = full_latlon[cursor: cursor + count]
    cursor += count

    if len(segment) < 2:
        continue

    folium.PolyLine(
        locations=segment,
        color=leg_color,
        weight=5,
        opacity=0.9,
        tooltip=f"Leg {i+1}"
    ).add_to(folium_map)

folium_map

Although the geometry is continuous, OSRM internally
divides the route into logical legs. These colored
segments allow us to visually inspect how the engine
constructed the final matched path across different
sections of the trip.

### With Radiuses

The `radiuses` parameter allows us to define the
expected GPS accuracy (in meters) for each input point.

By default, OSRM assumes a relatively small radius,
which means the engine expects the GPS points to be
very close to the actual road geometry.

In this example, we increase the radius to **20 meters**
for each point. This gives OSRM more flexibility when
matching noisy or imprecise GPS traces.

In [ ]:
import requests

max_points_to_use = 80
points_portion = points[:max_points_to_use]

# Build coordinates string (lon,lat)
coordinates = ";".join(
    f"{p.longitude},{p.latitude}" for p in points_portion
)

# Radius for each point (meters)
radiuses = ";".join(["20"] * len(points_portion))

url = f"{host}/{service}/{version}/{profile}/{coordinates}"

params = {
    "geometries": "geojson",
    "overview": "full",
    "radiuses": radiuses,
    "tidy": "true"
}

response = requests.get(url, params=params)
response.status_code

After increasing the radius, we inspect the
`confidence` value from the returned matching.

The confidence score indicates how well the GPS
trace fits the road network. A higher value means
a better match.

By relaxing the radius constraint, OSRM is able
to produce a more reliable alignment for imperfect
GPS data.

In [ ]:
snapped_route = response.json()
matched_route = snapped_route["matchings"][0]
matched_route["confidence"]

## Conclusion

In this notebook, we explored the OSRM **Match** service
and its role in aligning raw GPS traces with the
underlying road network.

We demonstrated:

- Parsing a GPX file and extracting GPS coordinates  
- Visualizing raw GPS points on an interactive map  
- Understanding OSRM request structure for the Match service  
- Sending match requests with and without radiuses  
- Extracting snapped route geometry from the `matchings` field  
- Drawing the corrected route on the map  
- Segmenting the matched route by legs using different colors  
- Evaluating the `confidence` score of the matched result  

Unlike the Route service, which computes paths between
clean waypoints, the Match service works with noisy,
real-world GPS traces and reconstructs the most
plausible path taken on the road network.

We observed how increasing the radius parameter
improves robustness when dealing with GPS inaccuracies,
and how the confidence score reflects the quality
of the alignment.

By combining GPX parsing, visualization, map matching,
leg analysis, and confidence evaluation, we built a
complete workflow for transforming raw trajectory
data into a structured, road-aligned route.

This approach is essential for applications such as
vehicle tracking, trajectory reconstruction,
fleet analytics, and mobility pattern analysis.